# This is an introductory Pytorch and Pytorch-Geometric Jupyter *notebook*


## Copyright 2022 Dr. George Papagiannakis, papagian@csd.uoc.gr
### All Rights Reserved

### University of Crete & Foundation for Research & Technology - Hellas (FORTH)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import sys
%matplotlib inline
import torch; torch.backends.mps.is_available()

# Installation instructions of pytorch and pytorch geometric
- https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
- for M1 macOS: https://github.com/rusty1s/pytorch_scatter/issues/241 and for torch-sparse:
    - `pip install git+https://github.com/rusty1s/pytorch_sparse.git `
    - `python -c "import torch; print(torch.__version__)"`

# Train and create our own GNN layer
## https://github.com/pyg-team/pytorch_geometric 

In [ ]:
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='.', name='Cora')

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = GCN(dataset.num_features, 16, dataset.num_classes)

In [ ]:
import torch
from torch import Tensor
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr="max")  # "Max" aggregation.
        self.mlp = Sequential(
            Linear(2 * in_channels, out_channels),
            ReLU(),
            Linear(out_channels, out_channels),
        )

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        return self.propagate(edge_index, x=x)  # shape [num_nodes, out_channels]

    def message(self, x_j: Tensor, x_i: Tensor) -> Tensor:
        # x_j: Source node features of shape [num_edges, in_channels]
        # x_i: Target node features of shape [num_edges, in_channels]
        edge_features = torch.cat([x_i, x_j - x_i], dim=-1)
        return self.mlp(edge_features)  # shape [num_edges, out_channels]

In [6]:
#pytorch-geometric example
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [7]:
Data(edge_index=[2, 4], x=[3, 1])

Data(x=[2], edge_index=[2])

### Load several networkx graphs in pytorch geometric and train a graph neural network in python to predict new graph embedding

To load several NetworkX graphs in PyTorch Geometric, you'll first need to convert them to PyTorch Geometric's data structure, which is called a Data object. This can be done by using the from_networkx method provided by PyTorch Geometric.

Here's an example of how you might load a list of NetworkX graphs and convert them to PyTorch Geometric's data structure:

In [ ]:
mport networkx as nx
from torch_geometric.data import Data
import torch_geometric.utils as tgu

graphs = [nx.barabasi_albert_graph(100, 5), nx.erdos_renyi_graph(100, 0.1)]

data_list = [tgu.from_networkx(G) for G in graphs]

To train a graph neural network on these graphs, you'll need to use a PyTorch Geometric model that is designed for this task. For example, you can use the GCN (Graph Convolutional Network) model, which is a simple and commonly used model for graph classification tasks.

Here's an example of how you might use the GCN model to train a graph neural network on the list of graphs:

In [ ]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.linear = Linear(output_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)

        x = self.linear(x)
        return x

model = GCN(input_dim=100, hidden_dim=128, output_dim=100)

Once we've defined your model, we can train it using the data_list you created earlier and the PyTorch optimizers like Adam, Adagrad and so on. The last layer of the model produces the new graph embedding as the output.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):
    for i, data in enumerate(data_list):
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.functional.mse_loss(output, data.x)
        loss.backward()
        optimizer.step()